In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.11.146.1-preview"

Installed Packages Microsoft.SemanticKernel, 0.11.146.1-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;

In [3]:
IKernel kernel = Kernel.Builder.Build();

In [4]:
kernel.Config.AddAzureOpenAITextCompletionService(
    "GPT3",      
    "text-davinci-003",                  
    "Your Endpoint",    
    "Your Key"          
);

In [5]:
string skPrompt = """

请帮我把 {{$input}} 进行类别确认，类别包括天气，课程，生成式，如果不太清楚，请回答没法确认，分类参考如下：

问: 会下雨吗？ 类别:天气
问: 今天温度？ 类别:天气
问: 适度多少？ 类别:天气
问: 什么是新能源车？ 类别: 课程
问: 电动车的特点 类别: 课程
问: 概念是什么？ 类别: 课程
问: 课程相关的内容有哪些？ 类别: 课程
问: 写一首诗歌？ 类别: 生成式
问: 翻译一下 类别: 生成式
问: 计算结果 类别: 生成式

如果能确认类别，天气相关请只输出 1 ， 课程相关请只输出 2 ， 生成式相关请只输出 3 ，没法确认相关请只输出 0，并把{{$input}}和它的类别参考以下 json 格式输出

{""question"":""{{$input}}"",""label"":""{{$label}}""}

""";

In [6]:
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
		MaxTokens = 60,
		FrequencyPenalty = (float)0,
		PresencePenalty = (float)0
    }
};

In [7]:
var promptTemplate = new PromptTemplate(
    skPrompt,                        // Prompt template defined in natural language
    promptConfig,                    // Prompt configuration
    kernel                           // SK instance
);

In [8]:
skPrompt


请帮我把 {{$input}} 进行类别确认，类别包括天气，课程，生成式，如果不太清楚，请回答没法确认，分类参考如下：

问: 会下雨吗？ 类别:天气
问: 今天温度？ 类别:天气
问: 适度多少？ 类别:天气
问: 什么是新能源车？ 类别: 课程
问: 电动车的特点 类别: 课程
问: 概念是什么？ 类别: 课程
问: 课程相关的内容有哪些？ 类别: 课程
问: 写一首诗歌？ 类别: 生成式
问: 翻译一下 类别: 生成式
问: 计算结果 类别: 生成式

如果能确认类别，天气相关请只输出 1 ， 课程相关请只输出 2 ， 生成式相关请只输出 3 ，没法确认相关请只输出 0，并把{{$input}}和它的类别参考以下 json 格式输出

{""question"":""{{$input}}"",""label"":""{{$label}}""}


In [9]:

var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

In [10]:
var classificationFunction = kernel.RegisterSemanticFunction("TextSkill", "TextClassification", functionConfig);

In [11]:
var input = """
今天广州天气怎么样？
""";

In [12]:
var classification = await kernel.RunAsync(input, classificationFunction);

Console.WriteLine(classification);


{"question": "今天广州天气怎么样？", "label": "1"}
